In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install ultralytics rasterio --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.7/645.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 48.4 MB/s eta 0:00:00


In [ ]:
%pip uninstall -y albumentations

Found existing installation: albumentations 1.3.1
Uninstalling albumentations-1.3.1:
  Successfully uninstalled albumentations-1.3.1


In [ ]:
from ultralytics import YOLO
import numpy as np
from PIL import Image
from skimage import draw
import random
from pathlib import Path
from shutil import move
from matplotlib import pyplot as plt
import torch
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = YOLO('yolov8n-seg.pt')  # load a pretrained model (recommended for training)

In [ ]:
!unzip drive/MyDrive/patches_train.zip

Streaming output truncated to the last 5000 lines.
  inflating: patches_train/masks/train_mask_12539.png  
  inflating: patches_train/masks/train_mask_25395.png  
  inflating: patches_train/masks/train_mask_13633.png  
  inflating: patches_train/masks/train_mask_15242.png  
  inflating: patches_train/masks/train_mask_7384.png  
  inflating: patches_train/masks/train_mask_23928.png  
  inflating: patches_train/masks/train_mask_25381.png  
  inflating: patches_train/masks/train_mask_11024.png  
  inflating: patches_train/masks/train_mask_26688.png  
  inflating: patches_train/masks/train_mask_19478.png  
  inflating: patches_train/masks/train_mask_18766.png  
  inflating: patches_train/masks/train_mask_26850.png  
  inflating: patches_train/masks/train_mask_17455.png  
  inflating: patches_train/masks/train_mask_5593.png  
  inflating: patches_train/masks/train_mask_27596.png  
  inflating: patches_train/masks/train_mask_16993.png  
  inflating: patches_train/masks/train_mask_4855.png  


In [ ]:
!mkdir dataset
!mkdir dataset/train
!mkdir dataset/train/images
!mkdir dataset/train/labels
!mkdir dataset/val
!mkdir dataset/val/images
!mkdir dataset/val/labels

In [ ]:
from rasterio import features, transform

def create_label(image_path, label_path):
    arr = np.asarray(Image.open(image_path))
    polygons = list(features.shapes(arr, mask=(arr > 0)))

    # Debug
    # img = features.rasterize(polygons, out_shape=(5000, 5000))
    # plt.imshow(img)

    with open(label_path, 'w') as f:
        for cords in polygons:
            cords = cords[0]['coordinates'][0]
            label_line = '0 ' + ' '.join([f'{int(cord[0])/arr.shape[0]} {int(cord[1])/arr.shape[1]}' for cord in cords]) + '\n'
            f.write(label_line)

n = 1

for img_path in tqdm(list(Path(f'patches_train/masks').iterdir())):
    if n%2 == 1:
        to = 'train'
    else:
        to = 'val'

    n += 1
    # print(img_path)

    if '.DS_Store' in str(img_path):
        continue

    label_path = f'dataset/{to}/labels/' + f'{img_path.stem}.txt'
    label_line = create_label(img_path, label_path)
    p = img_path.stem.replace('mask', 'image') + '.png'

    # try:
    move(f'patches_train/images/{p}', f'dataset/{to}/images/{img_path.stem + ".png"}')
    # except:
        # print('err')


Streaming output truncated to the last 5000 lines.
patches_train/masks/train_mask_22499.png
patches_train/masks/train_mask_20129.png
patches_train/masks/train_mask_15549.png
patches_train/masks/train_mask_17281.png
patches_train/masks/train_mask_13280.png
patches_train/masks/train_mask_9672.png
patches_train/masks/train_mask_18058.png
patches_train/masks/train_mask_27869.png
patches_train/masks/train_mask_16410.png
patches_train/masks/train_mask_21176.png
patches_train/masks/train_mask_14388.png
patches_train/masks/train_mask_14934.png
patches_train/masks/train_mask_14439.png
patches_train/masks/train_mask_12897.png
patches_train/masks/train_mask_9436.png
patches_train/masks/train_mask_6425.png
patches_train/masks/train_mask_14209.png
patches_train/masks/train_mask_6970.png
patches_train/masks/train_mask_5799.png
patches_train/masks/train_mask_22969.png
patches_train/masks/train_mask_14718.png
patches_train/masks/train_mask_30733.png
patches_train/masks/train_mask_7207.png
patches_trai

In [ ]:
%%writefile skoltech-seg.yaml
path: .  # dataset root dir
train: /content/dataset/train/images  # train images (relative to 'path') 4 images
val: /content/dataset/val/images  # val images (relative to 'path') 4 images
test:  # test images (optional)

# Classes (80 COCO classes)
names:
  0: building

In [ ]:
model.to(device)
model.train(data="skoltech-seg.yaml", epochs=20, imgsz=256, batch=256, close_mosaic=0, device=0)
# results = model.train(data='skoltech-seg.yaml', epochs=10, imgsz=256, device='gpu')

Ultralytics YOLOv8.0.217 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=skoltech-seg.yaml, epochs=20, patience=50, batch=256, imgsz=256, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=F

train: Scanning /content/dataset/train/labels.cache... 15438 images, 8386 backgrounds, 0 corrupt: 100%|██████████| 15438/15438 [00:00<?, ?it/s]
val: Scanning /content/dataset/val/labels.cache... 15438 images, 8341 backgrounds, 0 corrupt: 100%|██████████| 15438/15438 [00:00<?, ?it/s]


Plotting labels to runs/segment/train7/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.002), 76 bias(decay=0.0)
Image sizes 256 train, 256 val
Using 2 dataloader workers
Logging results to runs/segment/train7
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       1/20      12.5G      3.565      2.855      3.022      1.404         57        256: 100%|██████████| 61/61 [03:01<00:00,  2.97s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [02:06<00:00,  4.08s/it]


                   all      15438      13136   0.000384     0.0174   0.000196    2.6e-05    0.00866      0.393    0.00725    0.00192

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       2/20      12.5G      3.547      2.993      3.008      1.381        176        256:  46%|████▌     | 28/61 [01:16<01:21,  2.47s/it]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7930ace5f250>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7930ace5f250>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/l

                   all      15438      13136   0.000103    0.00221   8.17e-05   2.93e-05    0.00505      0.108    0.00282    0.00096

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       3/20      12.5G      3.569      3.341      3.054      1.375         53        256: 100%|██████████| 61/61 [02:55<00:00,  2.87s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [02:03<00:00,  3.97s/it]


                   all      15438      13136   0.000484      0.029   0.000476   7.02e-05    0.00651       0.39     0.0158    0.00365

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       4/20      12.5G      3.543      3.122      3.046      1.362        149        256:  26%|██▌       | 16/61 [00:41<02:06,  2.81s/it]

In [ ]:
move('/content/runs/segment/train6/weights/best.pt', '/content/drive/MyDrive/bbbbest.pt')

In [ ]:
from glob import glob

p = model.predict(glob('/content/dataset/val/images/*')[1])


image 1/1 /content/dataset/val/images/train_mask_9122.png: 256x256 (no detections), 17.2ms
Speed: 0.7ms preprocess, 17.2ms inference, 0.6ms postprocess per image at shape (1, 3, 256, 256)
